In [1]:
import urllib.request
from bs4 import BeautifulSoup

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_science_fiction_conventions'

In [23]:
ufile = urllib.request.urlopen(url)

In [24]:
newtext = str(ufile.read())

In [25]:
soup = BeautifulSoup(newtext,'lxml')

In [39]:
soup.th.next_element

'Convention'

In [41]:
match = soup.find_all('tr')

In [43]:
match[1]

<tr>\n<td><a href="/wiki/%C3%85con" title="\xc3\x85con">\xc3\x85con</a></td>\n<td><a href="/wiki/Mariehamn" title="Mariehamn">Mariehamn</a>, <a class="mw-redirect" href="/wiki/%C3%85land" title="\xc3\x85land">\xc3\x85land</a></td>\n<td>Finland</td>\n<td>SF/Fantasy literature</td>\n<td><a class="external autonumber" href="http://acon8.wordpress.com/" rel="nofollow">[1]</a></td>\n<td>2016-05-05</td>\n</tr>

In [45]:
matchsoup = BeautifulSoup(str(match[1]),'lxml')

In [51]:
matchsoup

<html><body><tr>\n<td><a href="/wiki/%C3%85con" title="\xc3\x85con">\xc3\x85con</a></td>\n<td><a href="/wiki/Mariehamn" title="Mariehamn">Mariehamn</a>, <a class="mw-redirect" href="/wiki/%C3%85land" title="\xc3\x85land">\xc3\x85land</a></td>\n<td>Finland</td>\n<td>SF/Fantasy literature</td>\n<td><a class="external autonumber" href="http://acon8.wordpress.com/" rel="nofollow">[1]</a></td>\n<td>2016-05-05</td>\n</tr></body></html>

In [52]:
tabmatch = matchsoup.find_all('td')

In [54]:
tabmatch

[<td><a href="/wiki/%C3%85con" title="\xc3\x85con">\xc3\x85con</a></td>,
 <td><a href="/wiki/Mariehamn" title="Mariehamn">Mariehamn</a>, <a class="mw-redirect" href="/wiki/%C3%85land" title="\xc3\x85land">\xc3\x85land</a></td>,
 <td>Finland</td>,
 <td>SF/Fantasy literature</td>,
 <td><a class="external autonumber" href="http://acon8.wordpress.com/" rel="nofollow">[1]</a></td>,
 <td>2016-05-05</td>]

Each row of the table has the same number of cells.  The country info is in cell[2].

In [59]:
tabmatch[2].next_element

'Finland'

In [60]:
if 'USA' in tabmatch[2]:
    print,"USA"